## Import libs

In [1]:
import os
from PIL import Image

## Convert labels to YOLO format

In [3]:
LABELS_DIR = './OID/Dataset/labels/val'  # folder with your current label .txt files
IMAGES_DIR = './OID/Dataset/images/val'  # where the matching images are
OUTPUT_DIR = './OID/Dataset/labels/yolo'  # where to save YOLO labels

os.makedirs(OUTPUT_DIR, exist_ok=True)

class_map = {'Egg': 0}

for label_file in os.listdir(LABELS_DIR):
    if not label_file.endswith('.txt'):
        continue

    label_path = os.path.join(LABELS_DIR, label_file)
    image_name = os.path.splitext(label_file)[0] + '.jpg'
    image_path = os.path.join(IMAGES_DIR, image_name)

    if not os.path.exists(image_path):
        print(f"Image not found: {image_name}, skipping")
        continue

    img = Image.open(image_path)
    img_width, img_height = img.size

    with open(label_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            print(f"Invalid line: {line}")
            continue

        class_name, x_min, y_min, x_max, y_max = parts
        class_id = class_map.get(class_name)
        if class_id is None:
            print(f"Unknown class: {class_name}")
            continue

        x_min, y_min, x_max, y_max = map(float, (x_min, y_min, x_max, y_max))
        x_center = (x_min + x_max) / 2 / img_width
        y_center = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height

        new_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
        new_lines.append(new_line)

    with open(os.path.join(OUTPUT_DIR, label_file), 'w') as f:
        f.write('\n'.join(new_lines))
